In [2]:
from nba_api.stats.static import players,teams
import pandas as pd
import requests
import sys
import os
import time
from datetime import datetime

def format_date_to_url(date):
    # Convert date from YYYYMMDD to datetime object
    date_obj = datetime.strptime(str(date), '%Y%m%d')
    
    # Format the date as MM%2FDD%2FYYYY
    formatted_date = date_obj.strftime('%m%%2F%d%%2F%Y')
    
    return formatted_date

def pull_data(url):


    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    json = requests.get(url,headers = headers).json()

    if len(json["resultSets"])== 1:

        
        data = json["resultSets"][0]["rowSet"]
        #print(data)
        columns = json["resultSets"][0]["headers"]
        #print(columns)
        
        df = pd.DataFrame.from_records(data, columns=columns)
    else:

        data = json["resultSets"]["rowSet"]
        
        #print(json)
        columns = json["resultSets"]["headers"][1]['columnNames']
        #print(columns)
        df = pd.DataFrame.from_records(data, columns=columns)

    time.sleep(.5)
    return df

# Example usage
def pull_game_level2(dates, unit='Player'):
    frames = []
    shotcolumns = ['FGA_FREQUENCY', 'FGM', 'FGA', 'FG_PCT', 'EFG_PCT', 'FG2A_FREQUENCY', 'FG2M', 'FG2A', 'FG2_PCT', 
                   'FG3A_FREQUENCY', 'FG3M', 'FG3A', 'FG3_PCT']
    
    for year in range(2014, 2025):
        year_frame = []
        year_dates = [date for date in dates if date > (year * 10000) and date < ((year + 1) * 10000)]
        season = str(year - 1) + '-' + str(year)[-2:]
        
        for date in year_dates:
            date_num = int(date)
            date = format_date_to_url(date)
            url=f"https://stats.nba.com/stats/leaguedashplayershotlocations?College=&Conference=&Country=&DateFrom={date}&DateTo={date}&DistanceRange=By%20Zone&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season}&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight="
            df=pull_data(url)

            zone_columns=['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'NICKNAME',
             'RA_FGM', 'RA_FGA', 'RA_FG_PCT',               # Restricted Area
             'ITP_FGM', 'ITP_FGA', 'ITP_FG_PCT',             # In The Paint (Non-RA)
             'MID_FGM', 'MID_FGA', 'MID_FG_PCT',             # Mid Range
             'LEFT_CORNER_3_FGM', 'LEFT_CORNER_3_FGA', 'LEFT_CORNER_3_FG_PCT',  # Left Corner 3
             'RIGHT_CORNER_3_FGM', 'RIGHT_CORNER_3_FGA', 'RIGHT_CORNER_3_FG_PCT', # Right Corner 3
      

                           # All Corner 3s
             'ABOVE_BREAK_3_FGM', 'ABOVE_BREAK_3_FGA', 'ABOVE_BREAK_3_FG_PCT', 
                   'BACKCOURT_FGM', 'BACKCOURT_FGA', 'BACKCOURT_FG_PCT', # Right Corner 3
                          
                          'CORNER_3_FGM', 'CORNER_3_FGA', 'CORNER_3_FG_PCT'  ]  # Above the Break 3

            df.columns=zone_columns
            url2=f"https://stats.nba.com/stats/leaguedashptdefend?College=&Conference=&Country=&DateFrom{date}=&DateTo={date}&DefenseCategory=Less%20Than%206Ft&Division=&DraftPick=&DraftYear=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&Season={season}&SeasonSegment=&SeasonType=Playoffs&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight="
            df2=pull_data(url2)
            df2.rename(columns={'CLOSE_DEF_PERSON_ID':'PLAYER_ID'},inplace=True)

            url3=f"https://stats.nba.com/stats/leaguedashplayershotlocations?College=&Conference=&Country=&DateFrom={date}&DateTo={date}&DistanceRange=5ft%20Range&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season}&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight="
            df3=pull_data(url3)
            df3.columns=['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBR', 'AGE', 'NICKNAME',
             'FGM_LT_5', 'FGA_LT_5', 'FGP_LT_5',      # Less than 5 feet
             'FGM_5_9', 'FGA_5_9', 'FGP_5_9',         # 5-9 feet
             'FGM_10_14', 'FGA_10_14', 'FGP_10_14',   # 10-14 feet
             'FGM_15_19', 'FGA_15_19', 'FGP_15_19',   # 15-19 feet
             'FGM_20_24', 'FGA_20_24', 'FGP_20_24',   # 20-24 feet
             'FGM_25_29', 'FGA_25_29', 'FGP_25_29',   # 25-29 feet
             'FGM_30_34', 'FGA_30_34', 'FGP_30_34',   # 30-34 feet
             'FGM_35_39', 'FGA_35_39', 'FGP_35_39',   # 35-39 feet
             'FGM_40_PLUS', 'FGA_40_PLUS', 'FGP_40_PLUS'  # 40+ feet
            ]
            frames = [df2,df3]
            for frame in frames:
                
                joined_columns = set(frame.columns) - set(df.columns)
                joined_columns = list(joined_columns)
                joined_columns.append('PLAYER_ID')
                frame = frame[joined_columns]

                df = df.merge(frame, on='PLAYER_ID',how='left').reset_index(drop=True)

            df['year'] = year
            df['date']=date_num
            print(date)

            year_frame.append(df)

        yeardata=pd.concat(year_frame)
        yeardata.to_csv(str(year)+'_2.csv',index=False)
        frames.append(yeardata)
        print(f"Year: {year}")

    total = pd.concat(frames)
    return total

def get_dates():
    dates=[]
    for year in range(2014,2026):
    
        for team in teams.get_teams():
            team_id=team['id']
            path = '../team/'+str(year)+'ps/'+str(team_id)+'.csv'
            if os.path.exists(path):
                df=pd.read_csv(path)
    
                df=df[['PLAYER_ID','HTM','VTM','GAME_DATE']]
                df.drop_duplicates(inplace=True)
                dates.append(df)
    return pd.concat(dates)
dateframe=get_dates()
dates=dateframe['GAME_DATE'].unique().tolist()

#df= pull_game_level(dates)
data=pull_game_level2(dates)

04%2F19%2F2014


KeyboardInterrupt: 